In [11]:
import boto3
import json

In [13]:
bucket = "awspersonalizegehealthcarev1"

In [7]:
# Let's use Amazon S3
s3 = boto3.resource('s3')

In [9]:
# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)

awspersonalizegehealthcarev1
cf-templates-a9whgn9nx91i-us-east-1
elasticbeanstalk-eu-west-1-416556863142


In [14]:
# Attach policy to s3 bucket
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': '8KTZMWEE7K12T6BZ',
  'HostId': 'zOAb3qex8oBHikpY3v3jEf09QQgqz7U0Yq8ovCTWdAg6TUCJeFYJWB3wHDBOoUPKvtXDwo3P33s=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'zOAb3qex8oBHikpY3v3jEf09QQgqz7U0Yq8ovCTWdAg6TUCJeFYJWB3wHDBOoUPKvtXDwo3P33s=',
   'x-amz-request-id': '8KTZMWEE7K12T6BZ',
   'date': 'Wed, 06 Oct 2021 10:15:33 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [15]:
# Create personalize role
iam = boto3.client("iam")

role_name = "PersonalizeRoleDemo"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)

NameError: name 'time' is not defined

In [16]:
import time
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::416556863142:role/PersonalizeRoleDemo
